# One Hot Encoder

## Introdução

Este notebook está na mesma categoria que o "LabelEncoder.ipynb" pois ambos versam sobre a codificação de categorias. No caso do label encoder, a gente transformava strings em números de modo que havia uma relação de ordem entre os elementos. Por exemplo, se temos strings "cartão gold" e "cartão black", é mais adequado atribuirmos um número x para o black e y para o gold de modo que x > y pois é bem sabido que um cartão de crédito black é mais valioso que um gold.

Contudo, ocorre ocasiões nas quais não há sentido em estabelecer uma relação de ordem entre as variáveis. Por exemplo, por que a cor "violeta" valeria menos ou mais do que a cor "azul"? Para sanar esse tipo de problema, usamos o *One Hot Encoding*, que transforma essas strings em vetores, e assim conseguimos transformar variáveis categóricas em numéricas de modo que o algoritmo de machine learning não tire conclusões errôneas.

## Dados Iniciais

Criaremos em essência o mesmo conjunto de dados, e a partir dele faremos um array e um DataFrame para mostrar que em ambas as tipagens o código funciona.

In [3]:
from numpy import array
from pandas import DataFrame

lista = ['vermelho', 'azul', 'rosa', 'azul', 'bege', 'vermelho']

dicionario = {'cores': lista}
df = DataFrame(data = dicionario)
df

,cores
0,vermelho
1,azul
2,rosa
3,azul
4,bege
5,vermelho


In [4]:
x = array(lista)
x

array(['vermelho', 'azul', 'rosa', 'azul', 'bege', 'vermelho'],
      dtype='<U8')

## ```OneHotEncoder```

Para fazermos essa codificação de categorias, importamos a classe ```OneHotEncoder``` de ```sklearn.preprocessing``` . Daí, é só criar uma instância, e colocar no parâmetro ```sparse``` como ```False``` para indicar que queremos um array - ao invés de matriz - depois de efetuar a codificação.

In [1]:
from sklearn.preprocessing import OneHotEncoder

codificador = OneHotEncoder(sparse = False)

Para fazermos a codificação de fato, é só usar o método ```fit_transform```, e colocar na entrada os dados a serem codificados. 

No caso de um ```DataFrame```, é só colocar diretamente no argumento do método.

In [5]:
u = codificador.fit_transform(df)
u

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

Já no caso de ```array```, precisamos redimensionar os dados, pois os dados não possuem uma coluna, como podemos ver abaixo.

In [6]:
x.shape

(6,)

São apenas 6 linhas e "zero" colunas. Para sanar esse problema, é só fazer

In [7]:
xx = x.reshape(len(x),1)

E agora sim procedemos da mesma maneira que no caso do ```DataFrame```.

In [8]:
v = codificador.fit_transform(xx)
v

array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]])

## Dummy Variable Trap

Para o caso acima, basta apenas retirar uma coluna e evitamos o "Dummy variable trap".

In [9]:
fff = v[:,1:]

fff

array([[0., 0., 1.],
       [0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

Com isso, retiramos a primeira coluna do array. Contudo, essa tarefa se torna mais difícil quando temos outras categorias, e queremos juntar tudo num mesmo conjunto de dados. Por exemplo, imagine que os dados a serem trabalhados sejam

In [10]:
nomes = ['Antonio', 'Alberto', 'Andreza', 'Anísio', 'André', 'Aline']

idades = array([12, 33, 20, 87, 9, 46])

dicionario2 = {'Nome': nomes, 'Cor': lista, 'Idade': idades}

df2 = DataFrame(dicionario2)

df2

,Nome,Cor,Idade
0,Antonio,vermelho,12
1,Alberto,azul,33
2,Andreza,rosa,20
3,Anísio,azul,87
4,André,bege,9
5,Aline,vermelho,46


Suponha que queiramos apenas realizar essa codificação apenas para as cores. Deste modo, teremos que aumentar a tabela de 3 para 6 colunas (2 + 4), e depois retirar uma delas para evitar a multicolinearidade, fazendo com que tenhamos no final com 5 colunas.

Primeiramente, criamos a instância da classe ```OneHotEncoder```, e além do parâmetro ```sparse```, inserimos também o parâmetro ```drop``` como ```'first'```, e assim o codificador já descarta automaticamente a primeira coluna depois do processo.

Assim, armazenamos o retorno do método ```fit_transform``` numa nova variável.

In [15]:
codificador2 = OneHotEncoder(sparse = False,
                             drop = 'first')



ArrayCores = codificador2.fit_transform(df2[['Cor']])

ArrayCores

array([[0., 0., 1.],
       [0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

Feito isso, transformamos esse array num DataFrame, para juntar os dados em seguida.

In [16]:
dfArrayCores = DataFrame(ArrayCores)

dfArrayCores

,0,1,2
0,0.0,0.0,1.0
1,0.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,0.0
4,1.0,0.0,0.0
5,0.0,0.0,1.0


Com o método ```join```, juntamos os dados ao DataFrame inicial.

In [17]:
df2 = df2.join(dfArrayCores)

df2

,Nome,Cor,Idade,0,1,2
0,Antonio,vermelho,12,0.0,0.0,1.0
1,Alberto,azul,33,0.0,0.0,0.0
2,Andreza,rosa,20,0.0,1.0,0.0
3,Anísio,azul,87,0.0,0.0,0.0
4,André,bege,9,1.0,0.0,0.0
5,Aline,vermelho,46,0.0,0.0,1.0


Em seguida, retiramos a coluna 'Cor' pois os dados codificados já estão nas últimas colunas.

In [18]:
df2.drop(columns = ['Cor'],
         inplace = True)

df2

,Nome,Idade,0,1,2
0,Antonio,12,0.0,0.0,1.0
1,Alberto,33,0.0,0.0,0.0
2,Andreza,20,0.0,1.0,0.0
3,Anísio,87,0.0,0.0,0.0
4,André,9,1.0,0.0,0.0
5,Aline,46,0.0,0.0,1.0


Por fim, vamos pegar só as células propriamente ditas, pois em aprendizado de máquina trabalhamos dessa forma, e ficamos com

In [19]:
dados = df2.iloc[:,:].values

dados

array([['Antonio', 12, 0.0, 0.0, 1.0],
       ['Alberto', 33, 0.0, 0.0, 0.0],
       ['Andreza', 20, 0.0, 1.0, 0.0],
       ['Anísio', 87, 0.0, 0.0, 0.0],
       ['André', 9, 1.0, 0.0, 0.0],
       ['Aline', 46, 0.0, 0.0, 1.0]], dtype=object)